In [1]:
import numpy as np
import pandas as pd
import polars as pl
import sys
from natsort import natsorted
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
pd.set_option('display.max_columns',None)
import psycopg2
#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
import warnings
warnings.filterwarnings("ignore")
#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [67]:
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/WG_100Kb_Bins.xlsx')
df.columns = ['chrom', 'start', 'end']
print(df.shape)
df.head(3)

(27267, 3)


,chrom,start,end
0,chr1,0,100000
1,chr1,100000,200000
2,chr1,200000,300000


In [68]:
data = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/main_data_new_02_02_2024.xlsx')
print(data.shape)
data.head()

(14438, 4)


,chrom,start,end,type
0,chr1,3010218,3087058,cLAD
1,chr1,3126730,3196670,cLAD
2,chr1,3220610,3282860,cLAD
3,chr1,3339298,3357024,cLAD
4,chr1,3371740,3389850,cLAD


In [69]:
dataset1 = pd.DataFrame(df)
dataset2 = pd.DataFrame(data)

# Initialize an empty result DataFrame to store the matching regions
result_df = pd.DataFrame(columns=['chrom', 'start', 'end', 'start_df1', 'end_df1'])

# Iterate over each row in dataset2
for index2, row2 in dataset2.iterrows():
    # Find the matching rows in dataset1 based on chrom and start/end conditions
    matching_rows = dataset1[(dataset1['chrom'] == row2['chrom']) & ((dataset1['start'] <= row2['start']) & (row2['start'] <= dataset1['end']) | (dataset1['start'] <= row2['end']) & (row2['end'] <= dataset1['end']))]
    
    # Create new rows in the result DataFrame for each match
    for index1, matching_row in matching_rows.iterrows():
        result_df = result_df.append({'chrom': matching_row['chrom'],
                                      'start': row2['start'],
                                      'end': row2['end'],
                                      'start_df1': matching_row['start'],
                                      'end_df1': matching_row['end']},
                                     ignore_index=True)

# Display the result DataFrame
result_df

,chrom,start,end,start_df1,end_df1
0,chr1,3010218,3087058,3000000,3100000
1,chr1,3126730,3196670,3100000,3200000
2,chr1,3220610,3282860,3200000,3300000
3,chr1,3339298,3357024,3300000,3400000
4,chr1,3371740,3389850,3300000,3400000
...,...,...,...,...,...
22801,chrX,164898980,164923418,164900000,165000000
22802,chrX,164928258,165004632,164900000,165000000
22803,chrX,164928258,165004632,165000000,165100000
22804,chrX,165323428,165367578,165300000,165400000


In [70]:
result_df[result_df['start'] == 26864156]

,chrom,start,end,start_df1,end_df1
151,chr1,26864156,28041584,26800000,26900000
152,chr1,26864156,28041584,28000000,28100000


In [71]:
result_df[result_df['start_df1'] == 20700000]

,chrom,start,end,start_df1,end_df1
1476,chr13,20728934,20773990,20700000,20800000
2959,chr19,20656031,20706751,20700000,20800000
2960,chr19,20738413,20748913,20700000,20800000
3580,chr3,20667189,20725077,20700000,20800000
3581,chr3,20736593,20816019,20700000,20800000
4086,chr4,20784746,20970302,20700000,20800000
5033,chr7,20790396,23414102,20700000,20800000
10029,chr5,20690933,20706873,20700000,20800000
10030,chr5,20742003,20757003,20700000,20800000
14272,chr13,20309793,20728933,20700000,20800000


In [37]:
main.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/data.xlsx', index=False)

In [72]:
main = pd.merge(result_df, data, on = ['chrom', 'start', 'end'], how = 'outer', sort=False)
main = main[['chrom', 'start', 'end', 'type', 'start_df1', 'end_df1']]
main

,chrom,start,end,type,start_df1,end_df1
0,chr1,3010218,3087058,cLAD,3000000,3100000
1,chr1,3126730,3196670,cLAD,3100000,3200000
2,chr1,3220610,3282860,cLAD,3200000,3300000
3,chr1,3339298,3357024,cLAD,3300000,3400000
4,chr1,3371740,3389850,cLAD,3300000,3400000
...,...,...,...,...,...,...
22801,chrX,164898980,164923418,fLAD,164900000,165000000
22802,chrX,164928258,165004632,fLAD,164900000,165000000
22803,chrX,164928258,165004632,fLAD,165000000,165100000
22804,chrX,165323428,165367578,fLAD,165300000,165400000


In [73]:
main[main['start'] == 20686631]

,chrom,start,end,type,start_df1,end_df1
11922,chr1,20686631,20800287,fLAD,20600000,20700000
11923,chr1,20686631,20800287,fLAD,20800000,20900000


In [74]:
main[main['start_df1'] == 20700000]

,chrom,start,end,type,start_df1,end_df1
1476,chr13,20728934,20773990,cLAD,20700000,20800000
2959,chr19,20656031,20706751,cLAD,20700000,20800000
2960,chr19,20738413,20748913,cLAD,20700000,20800000
3580,chr3,20667189,20725077,cLAD,20700000,20800000
3581,chr3,20736593,20816019,cLAD,20700000,20800000
4086,chr4,20784746,20970302,cLAD,20700000,20800000
5033,chr7,20790396,23414102,cLAD,20700000,20800000
10029,chr5,20690933,20706873,ciLAD,20700000,20800000
10030,chr5,20742003,20757003,ciLAD,20700000,20800000
14272,chr13,20309793,20728933,fLAD,20700000,20800000


In [75]:
main['Difference'] = main['end'] - main['start']
main

,chrom,start,end,type,start_df1,end_df1,Difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840
1,chr1,3126730,3196670,cLAD,3100000,3200000,69940
2,chr1,3220610,3282860,cLAD,3200000,3300000,62250
3,chr1,3339298,3357024,cLAD,3300000,3400000,17726
4,chr1,3371740,3389850,cLAD,3300000,3400000,18110
...,...,...,...,...,...,...,...
22801,chrX,164898980,164923418,fLAD,164900000,165000000,24438
22802,chrX,164928258,165004632,fLAD,164900000,165000000,76374
22803,chrX,164928258,165004632,fLAD,165000000,165100000,76374
22804,chrX,165323428,165367578,fLAD,165300000,165400000,44150


In [76]:
main['concat_new'] = main['chrom'].astype('str') + '_' + main['start_df1'].astype('str') + '_' + main['end_df1'].astype('str')
main

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000
1,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000
2,chr1,3220610,3282860,cLAD,3200000,3300000,62250,chr1_3200000_3300000
3,chr1,3339298,3357024,cLAD,3300000,3400000,17726,chr1_3300000_3400000
4,chr1,3371740,3389850,cLAD,3300000,3400000,18110,chr1_3300000_3400000
...,...,...,...,...,...,...,...,...
22801,chrX,164898980,164923418,fLAD,164900000,165000000,24438,chrX_164900000_165000000
22802,chrX,164928258,165004632,fLAD,164900000,165000000,76374,chrX_164900000_165000000
22803,chrX,164928258,165004632,fLAD,165000000,165100000,76374,chrX_165000000_165100000
22804,chrX,165323428,165367578,fLAD,165300000,165400000,44150,chrX_165300000_165400000


In [77]:
df_new = main.copy()

In [78]:
df = df_new.copy()
df['difference'] = df.apply(lambda row: len(range(max(row['start'], row['start_df1']), min(row['end'], row['end_df1']))), axis=1)
df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
1,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
2,chr1,3220610,3282860,cLAD,3200000,3300000,62250,chr1_3200000_3300000,62250
3,chr1,3339298,3357024,cLAD,3300000,3400000,17726,chr1_3300000_3400000,17726
4,chr1,3371740,3389850,cLAD,3300000,3400000,18110,chr1_3300000_3400000,18110
...,...,...,...,...,...,...,...,...,...
22801,chrX,164898980,164923418,fLAD,164900000,165000000,24438,chrX_164900000_165000000,23418
22802,chrX,164928258,165004632,fLAD,164900000,165000000,76374,chrX_164900000_165000000,71742
22803,chrX,164928258,165004632,fLAD,165000000,165100000,76374,chrX_165000000_165100000,4632
22804,chrX,165323428,165367578,fLAD,165300000,165400000,44150,chrX_165300000_165400000,44150


In [79]:
df[df['start'] == 20686631]

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
11922,chr1,20686631,20800287,fLAD,20600000,20700000,113656,chr1_20600000_20700000,13369
11923,chr1,20686631,20800287,fLAD,20800000,20900000,113656,chr1_20800000_20900000,287


In [91]:
#df = df_new.copy()
# Function to impute missing rows
def impute_missing_rows(df):
    result = pd.DataFrame(columns=df.columns)
    
    for chrom, group in df.groupby('chrom'):
        start = group['start_df1'].min()
        end = group['end_df1'].max()
        
        # Generate new rows based on 100000 difference
        new_rows = pd.DataFrame({
            'start_df1': range(start, end, 100000),
            'end_df1': range(start + 100000, end + 100000, 100000),
            'chrom': [chrom]*len(range(start, end, 100000))
        })
        
        result = pd.concat([result, new_rows], ignore_index=True)
    
    return result

# Concatenate the original DataFrame with the imputed rows
result_df = pd.concat([df, impute_missing_rows(df)], ignore_index=True)

# Sort the DataFrame based on 'chrom' and 'start_df1'
result_df = result_df.sort_values(by=['chrom', 'start_df1']).reset_index(drop=True)
#result_df.drop_duplicates(subset=['start_df1', 'end_df1', 'chrom'], inplace=True)
result_df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
1,chr1,3087059,3126729,fLAD,3000000,3100000,39670,chr1_3000000_3100000,12941
2,chr1,NaN,NaN,NaN,3000000,3100000,NaN,NaN,NaN
3,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
4,chr1,3087059,3126729,fLAD,3100000,3200000,39670,chr1_3100000_3200000,26729
...,...,...,...,...,...,...,...,...,...
48238,chrX,166163391,166206469,ciLAD,166100000,166200000,43078,chrX_166100000_166200000,36609
48239,chrX,166142678,166163390,fLAD,166100000,166200000,20712,chrX_166100000_166200000,20712
48240,chrX,NaN,NaN,NaN,166100000,166200000,NaN,NaN,NaN
48241,chrX,166163391,166206469,ciLAD,166200000,166300000,43078,chrX_166200000_166300000,6469


In [92]:
cols = ['start', 'end', 'type', 'Difference']
result_df.loc[:,cols] = result_df.loc[:,cols].ffill()
result_df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
1,chr1,3087059,3126729,fLAD,3000000,3100000,39670,chr1_3000000_3100000,12941
2,chr1,3087059,3126729,fLAD,3000000,3100000,39670,NaN,NaN
3,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
4,chr1,3087059,3126729,fLAD,3100000,3200000,39670,chr1_3100000_3200000,26729
...,...,...,...,...,...,...,...,...,...
48238,chrX,166163391,166206469,ciLAD,166100000,166200000,43078,chrX_166100000_166200000,36609
48239,chrX,166142678,166163390,fLAD,166100000,166200000,20712,chrX_166100000_166200000,20712
48240,chrX,166142678,166163390,fLAD,166100000,166200000,20712,NaN,NaN
48241,chrX,166163391,166206469,ciLAD,166200000,166300000,43078,chrX_166200000_166300000,6469


In [95]:
result_df['concat_new'] = result_df['chrom'].astype('str') + '_' + result_df['start_df1'].astype('str') + '_' + result_df['end_df1'].astype('str')
result_df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
1,chr1,3087059,3126729,fLAD,3000000,3100000,39670,chr1_3000000_3100000,12941
2,chr1,3087059,3126729,fLAD,3000000,3100000,39670,chr1_3000000_3100000,NaN
3,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
4,chr1,3087059,3126729,fLAD,3100000,3200000,39670,chr1_3100000_3200000,26729
...,...,...,...,...,...,...,...,...,...
48238,chrX,166163391,166206469,ciLAD,166100000,166200000,43078,chrX_166100000_166200000,36609
48239,chrX,166142678,166163390,fLAD,166100000,166200000,20712,chrX_166100000_166200000,20712
48240,chrX,166142678,166163390,fLAD,166100000,166200000,20712,chrX_166100000_166200000,NaN
48241,chrX,166163391,166206469,ciLAD,166200000,166300000,43078,chrX_166200000_166300000,6469


In [96]:
result_df[result_df['start'] == 20686631]

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
493,chr1,20686631,20800287,fLAD,20600000,20700000,113656,chr1_20600000_20700000,13369
494,chr1,20686631,20800287,fLAD,20600000,20700000,113656,chr1_20600000_20700000,NaN
495,chr1,20686631,20800287,fLAD,20700000,20800000,113656,chr1_20700000_20800000,NaN
497,chr1,20686631,20800287,fLAD,20800000,20900000,113656,chr1_20800000_20900000,287
498,chr1,20686631,20800287,fLAD,20800000,20900000,113656,chr1_20800000_20900000,NaN
499,chr1,20686631,20800287,fLAD,20900000,21000000,113656,chr1_20900000_21000000,NaN
500,chr1,20686631,20800287,fLAD,21000000,21100000,113656,chr1_21000000_21100000,NaN


In [93]:
result_df[result_df['start'] == 20686631]

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
493,chr1,20686631,20800287,fLAD,20600000,20700000,113656,chr1_20600000_20700000,13369
494,chr1,20686631,20800287,fLAD,20600000,20700000,113656,NaN,NaN
495,chr1,20686631,20800287,fLAD,20700000,20800000,113656,NaN,NaN
497,chr1,20686631,20800287,fLAD,20800000,20900000,113656,chr1_20800000_20900000,287
498,chr1,20686631,20800287,fLAD,20800000,20900000,113656,NaN,NaN
499,chr1,20686631,20800287,fLAD,20900000,21000000,113656,NaN,NaN
500,chr1,20686631,20800287,fLAD,21000000,21100000,113656,NaN,NaN


In [90]:
result_df[result_df['start'] == 20686631]

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference


In [83]:
result_df['difference'].fillna(0, inplace=True)
result_df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
1,chr1,3087059,3126729,fLAD,3000000,3100000,39670,chr1_3000000_3100000,12941
2,chr1,3087059,3126729,fLAD,3000000,3100000,39670,NaN,0
3,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
4,chr1,3087059,3126729,fLAD,3100000,3200000,39670,chr1_3100000_3200000,26729
...,...,...,...,...,...,...,...,...,...
48238,chrX,166163391,166206469,ciLAD,166100000,166200000,43078,chrX_166100000_166200000,36609
48239,chrX,166142678,166163390,fLAD,166100000,166200000,20712,chrX_166100000_166200000,20712
48240,chrX,166142678,166163390,fLAD,166100000,166200000,20712,NaN,0
48241,chrX,166163391,166206469,ciLAD,166200000,166300000,43078,chrX_166200000_166300000,6469


In [84]:
result_df[result_df['difference'] == 0]

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
2,chr1,3087059,3126729,fLAD,3000000,3100000,39670,NaN,0
6,chr1,3196671,3220609,fLAD,3100000,3200000,23938,NaN,0
10,chr1,3282861,3339297,fLAD,3200000,3300000,56436,NaN,0
16,chr1,3389851,3420829,fLAD,3300000,3400000,30978,NaN,0
19,chr1,3389851,3420829,fLAD,3400000,3500000,30978,NaN,0
...,...,...,...,...,...,...,...,...,...
48234,chrX,165323428,165367578,fLAD,165800000,165900000,44150,NaN,0
48235,chrX,165323428,165367578,fLAD,165900000,166000000,44150,NaN,0
48236,chrX,165323428,165367578,fLAD,166000000,166100000,44150,NaN,0
48240,chrX,166142678,166163390,fLAD,166100000,166200000,20712,NaN,0


In [87]:
result_df[result_df['start'] == 166163391]

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
48238,chrX,166163391,166206469,ciLAD,166100000,166200000,43078,chrX_166100000_166200000,36609
48241,chrX,166163391,166206469,ciLAD,166200000,166300000,43078,chrX_166200000_166300000,6469
48242,chrX,166163391,166206469,ciLAD,166200000,166300000,43078,NaN,0


In [63]:
df_new = result_df.copy()
df_new['concat_new'] = pd.Categorical(df_new['concat_new'])
df_new = df_new.loc[df_new.groupby('concat_new')['difference'].idxmax()]
df_new

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
5607,chr10,100044008,100183876,fLAD,100000000,100100000,139868,chr10_100000000_100100000,55992
5610,chr10,100044008,100183876,fLAD,100100000,100200000,139868,chr10_100100000_100200000,83876
5615,chr10,100443718,100508304,fLAD,100400000,100500000,64586,chr10_100400000_100500000,56282
5617,chr10,100508305,100564187,cLAD,100500000,100600000,55882,chr10_100500000_100600000,55882
5623,chr10,100625046,100724954,fLAD,100600000,100700000,99908,chr10_100600000_100700000,74954
...,...,...,...,...,...,...,...,...,...
46930,chrX,98591751,98886751,fLAD,98800000,98900000,295000,chrX_98800000_98900000,86751
46940,chrX,99613243,99790011,fLAD,99600000,99700000,176768,chrX_99600000_99700000,86757
46943,chrX,99613243,99790011,fLAD,99700000,99800000,176768,chrX_99700000_99800000,90011
46947,chrX,99797273,99893911,fLAD,99800000,99900000,96638,chrX_99800000_99900000,93911


In [65]:
df_new[df_new['start_df1'] == 20700000]

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
10890,chr13,20728934,20773990,cLAD,20700000,20800000,45056,chr13_20700000_20800000,45056
22605,chr19,20748914,20882660,fLAD,20700000,20800000,133746,chr19_20700000_20800000,51086
27123,chr3,20736593,20816019,cLAD,20700000,20800000,79426,chr3_20700000_20800000,63407
30294,chr4,20662557,20784745,fLAD,20700000,20800000,122188,chr4_20700000_20800000,84745
32939,chr5,20757004,20813452,fLAD,20700000,20800000,56448,chr5_20700000_20800000,42996
38232,chr7,19970053,20790395,fLAD,20700000,20800000,820342,chr7_20700000_20800000,90395


In [27]:
df = result_df.copy()
df['difference'] = df.apply(lambda row: len(range(max(row['start'], row['start_df1']), min(row['end'], row['end_df1']))), axis=1)
df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
3,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
7,chr1,3220610,3282860,cLAD,3200000,3300000,62250,chr1_3200000_3300000,62250
11,chr1,3339298,3357024,cLAD,3300000,3400000,17726,chr1_3300000_3400000,17726
17,chr1,3420830,3664260,cLAD,3400000,3500000,243430,chr1_3400000_3500000,79170
...,...,...,...,...,...,...,...,...,...
48234,chrX,165004633,165323427,cLAD,165800000,165900000,318794,NaN,0
48235,chrX,165004633,165323427,cLAD,165900000,166000000,318794,NaN,0
48236,chrX,165004633,165323427,cLAD,166000000,166100000,318794,NaN,0
48237,chrX,165367579,166142677,cLAD,166100000,166200000,775098,chrX_166100000_166200000,42677


In [28]:
df_new = df.copy()
df_new['concat_new'] = pd.Categorical(df_new['concat_new'])
df_new = df_new.loc[df_new.groupby('concat_new')['difference'].idxmax()]
df_new

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
5605,chr10,100010737,100044007,ciLAD,100000000,100100000,33270,chr10_100000000_100100000,33270
5609,chr10,100183877,100443717,cLAD,100100000,100200000,259840,chr10_100100000_100200000,16123
5614,chr10,100183877,100443717,cLAD,100400000,100500000,259840,chr10_100400000_100500000,43717
5617,chr10,100508305,100564187,cLAD,100500000,100600000,55882,chr10_100500000_100600000,55882
5622,chr10,100596047,100625045,cLAD,100600000,100700000,28998,chr10_100600000_100700000,25045
...,...,...,...,...,...,...,...,...,...
46929,chrX,98886752,99613242,ciLAD,98800000,98900000,726490,chrX_98800000_98900000,13248
46939,chrX,98886752,99613242,ciLAD,99600000,99700000,726490,chrX_99600000_99700000,13242
46942,chrX,99790012,99797272,cLAD,99700000,99800000,7260,chrX_99700000_99800000,7260
46946,chrX,99893912,99959402,cLAD,99800000,99900000,65490,chrX_99800000_99900000,6088


In [30]:
df_new[df_new['start_df1'] == 20700000]

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
10890,chr13,20728934,20773990,cLAD,20700000,20800000,45056,chr13_20700000_20800000,45056
22602,chr19,20656031,20706751,cLAD,20700000,20800000,50720,chr19_20700000_20800000,6751
27122,chr3,20667189,20725077,cLAD,20700000,20800000,57888,chr3_20700000_20800000,25077
30293,chr4,20784746,20970302,cLAD,20700000,20800000,185556,chr4_20700000_20800000,15254
32936,chr5,20690933,20706873,ciLAD,20700000,20800000,15940,chr5_20700000_20800000,6873
38231,chr7,20790396,23414102,cLAD,20700000,20800000,2623706,chr7_20700000_20800000,9604


In [13]:
# Function to impute missing rows
def impute_missing_rows(df):
    result = pd.DataFrame(columns=df.columns)
    
    for chrom, group in df.groupby('chrom'):
        start = group['start_df1'].min()
        end = group['end_df1'].max()
        
        # Generate new rows based on 100000 difference
        new_rows = pd.DataFrame({
            'start_df1': range(start, end, 100000),
            'end_df1': range(start + 100000, end + 100000, 100000),
            'chrom': [chrom]*len(range(start, end, 100000))
        })
        
        result = pd.concat([result, new_rows], ignore_index=True)
    
    return result

# Concatenate the original DataFrame with the imputed rows
result_df = pd.concat([df, impute_missing_rows(df)], ignore_index=True)

# Sort the DataFrame based on 'chrom' and 'start_df1'
result_df = result_df.sort_values(by=['chrom', 'start_df1']).reset_index(drop=True)
result_df.drop_duplicates(subset=['start_df1', 'end_df1', 'chrom'], inplace=True)
result_df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
3,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
7,chr1,3220610,3282860,cLAD,3200000,3300000,62250,chr1_3200000_3300000,62250
11,chr1,3339298,3357024,cLAD,3300000,3400000,17726,chr1_3300000_3400000,17726
17,chr1,3420830,3664260,cLAD,3400000,3500000,243430,chr1_3400000_3500000,79170
...,...,...,...,...,...,...,...,...,...
48234,chrX,NaN,NaN,NaN,165800000,165900000,NaN,NaN,NaN
48235,chrX,NaN,NaN,NaN,165900000,166000000,NaN,NaN,NaN
48236,chrX,NaN,NaN,NaN,166000000,166100000,NaN,NaN,NaN
48237,chrX,165367579,166142677,cLAD,166100000,166200000,775098,chrX_166100000_166200000,42677


In [15]:
cols = ['start', 'end', 'type', 'Difference']
result_df.loc[:,cols] = result_df.loc[:,cols].ffill()
result_df

,chrom,start,end,type,start_df1,end_df1,Difference,concat_new,difference
0,chr1,3010218,3087058,cLAD,3000000,3100000,76840,chr1_3000000_3100000,76840
3,chr1,3126730,3196670,cLAD,3100000,3200000,69940,chr1_3100000_3200000,69940
7,chr1,3220610,3282860,cLAD,3200000,3300000,62250,chr1_3200000_3300000,62250
11,chr1,3339298,3357024,cLAD,3300000,3400000,17726,chr1_3300000_3400000,17726
17,chr1,3420830,3664260,cLAD,3400000,3500000,243430,chr1_3400000_3500000,79170
...,...,...,...,...,...,...,...,...,...
48234,chrX,165004633,165323427,cLAD,165800000,165900000,318794,NaN,NaN
48235,chrX,165004633,165323427,cLAD,165900000,166000000,318794,NaN,NaN
48236,chrX,165004633,165323427,cLAD,166000000,166100000,318794,NaN,NaN
48237,chrX,165367579,166142677,cLAD,166100000,166200000,775098,chrX_166100000_166200000,42677


In [16]:
df_new = result_df.copy()
df_new['concat_new'] = pd.Categorical(df_new['concat_new'])
df_new = df_new.loc[df_new.groupby('concat_new')['difference'].idxmax()]
df_new

TypeError: reduction operation 'argmax' not allowed for this dtype

In [111]:
df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/WG_100Kb_Bins_ci_c_fLADs_Overlap_02_02_2024.bed', sep = '\t', header=None)
df.columns = ['chrom', 'start', 'end', 'chrom_ci', 'start_ci', 'end_ci', 'stand1', 'diff_ci', 'chrom_c', 'start_c', 'end_c', 'stand2', 'diff_c', 'chrom_f', 'start_f', 'end_f', 'stand3', 'diff_f' ]
df['concat'] = df['chrom'].astype('str') + '_' + df['start'].astype('str') + '_' + df['end'].astype('str')
df

,chrom,start,end,chrom_ci,start_ci,end_ci,stand1,diff_ci,chrom_c,start_c,end_c,stand2,diff_c,chrom_f,start_f,end_f,stand3,diff_f,concat
0,chr1,0,100000,.,-1,-1,.,0,.,-1,-1,.,0,.,-1,-1,.,0,chr1_0_100000
1,chr1,100000,200000,.,-1,-1,.,0,.,-1,-1,.,0,.,-1,-1,.,0,chr1_100000_200000
2,chr1,200000,300000,.,-1,-1,.,0,.,-1,-1,.,0,.,-1,-1,.,0,chr1_200000_300000
3,chr1,300000,400000,.,-1,-1,.,0,.,-1,-1,.,0,.,-1,-1,.,0,chr1_300000_400000
4,chr1,400000,500000,.,-1,-1,.,0,.,-1,-1,.,0,.,-1,-1,.,0,chr1_400000_500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33842,chrY,91300000,91400000,.,-1,-1,.,0,.,-1,-1,.,0,.,-1,-1,.,0,chrY_91300000_91400000
33843,chrY,91400000,91500000,.,-1,-1,.,0,.,-1,-1,.,0,.,-1,-1,.,0,chrY_91400000_91500000
33844,chrY,91500000,91600000,.,-1,-1,.,0,.,-1,-1,.,0,.,-1,-1,.,0,chrY_91500000_91600000
33845,chrY,91600000,91700000,.,-1,-1,.,0,.,-1,-1,.,0,.,-1,-1,.,0,chrY_91600000_91700000


In [112]:
# Calculate the sum of unique values in 'Value1', 'Value2', and 'Value3' for each group
grouped_df = df.groupby('concat').agg({
    'diff_ci': lambda x: pd.Series.unique(x).sum(),
    'diff_c': lambda x: pd.Series.unique(x).sum(),
    'diff_f': lambda x: pd.Series.unique(x).sum()
}).reset_index()
grouped_df

,concat,diff_ci,diff_c,diff_f
0,chr10_0_100000,0,0,0
1,chr10_100000000_100100000,33270,0,66728
2,chr10_10000000_10100000,0,100000,0
3,chr10_1000000_1100000,0,0,0
4,chr10_100000_200000,0,0,0
...,...,...,...,...
27262,chrY_9500000_9600000,0,0,0
27263,chrY_9600000_9700000,0,0,0
27264,chrY_9700000_9800000,0,0,0
27265,chrY_9800000_9900000,0,0,0


In [113]:
grouped_df['Max_Column'] = grouped_df[['diff_ci', 'diff_c', 'diff_f']].idxmax(axis=1)
grouped_df.loc[(grouped_df[['diff_ci', 'diff_c', 'diff_f']] == 0).all(axis=1), 'Max_Column'] = 'No_LAD'
grouped_df

,concat,diff_ci,diff_c,diff_f,Max_Column
0,chr10_0_100000,0,0,0,No_LAD
1,chr10_100000000_100100000,33270,0,66728,diff_f
2,chr10_10000000_10100000,0,100000,0,diff_c
3,chr10_1000000_1100000,0,0,0,No_LAD
4,chr10_100000_200000,0,0,0,No_LAD
...,...,...,...,...,...
27262,chrY_9500000_9600000,0,0,0,No_LAD
27263,chrY_9600000_9700000,0,0,0,No_LAD
27264,chrY_9700000_9800000,0,0,0,No_LAD
27265,chrY_9800000_9900000,0,0,0,No_LAD


In [114]:
grouped_df['chrom'] = grouped_df['concat'].str.split('_').str[0]
grouped_df['start'] = grouped_df['concat'].str.split('_').str[1]
grouped_df['end'] = grouped_df['concat'].str.split('_').str[2]
grouped_df

,concat,diff_ci,diff_c,diff_f,Max_Column,chrom,start,end
0,chr10_0_100000,0,0,0,No_LAD,chr10,0,100000
1,chr10_100000000_100100000,33270,0,66728,diff_f,chr10,100000000,100100000
2,chr10_10000000_10100000,0,100000,0,diff_c,chr10,10000000,10100000
3,chr10_1000000_1100000,0,0,0,No_LAD,chr10,1000000,1100000
4,chr10_100000_200000,0,0,0,No_LAD,chr10,100000,200000
...,...,...,...,...,...,...,...,...
27262,chrY_9500000_9600000,0,0,0,No_LAD,chrY,9500000,9600000
27263,chrY_9600000_9700000,0,0,0,No_LAD,chrY,9600000,9700000
27264,chrY_9700000_9800000,0,0,0,No_LAD,chrY,9700000,9800000
27265,chrY_9800000_9900000,0,0,0,No_LAD,chrY,9800000,9900000


In [115]:
grouped_df.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Positions_extensions/Latest_data/count_data.xlsx', index=False)